In [22]:
import osmnx as ox
from shapely.geometry import LineString, shape
import sys
import geopandas as gpd
import pandas as pd
from tqdm import tqdm
import numpy as np
sys.path.append("../src/")
from utils import get_gdf


## Carga de grafos y dataset de árboles

In [23]:
graph_walk = ox.load_graphml("../data/valencia_walking_network.graphml")
graph_cycle = ox.load_graphml("../data/valencia_cycling_network.graphml")

In [24]:
arboles = pd.read_csv("../data/arbratge-arbolado.csv", sep=';')

arboles["geometry"] = arboles["geo_shape"].apply(
    lambda x: shape(eval(x)))

gdf_arboles = get_gdf(arboles)

## Prueba con subset

In [25]:
#Prueba con un subset para ver que funciona

graph_proj = ox.project_graph(graph_walk, to_crs="EPSG:25830")
edges = ox.graph_to_gdfs(graph_proj, nodes=False)

# Proyectar árboles
arboles_proj = arboles.copy()
arboles_proj = gpd.GeoDataFrame(arboles_proj, geometry="geometry", crs="EPSG:4326")
arboles_proj = arboles_proj.to_crs(edges.crs)

# Crear índice espacial para eficiencia
sindex = arboles_proj.sindex

# Parámetros
radio = 15  # metros
w_len, w_sombra = 0.2, 0.8  # pesos para temperatura 25-30 °C

# Calcular número de árboles cercanos
def contar_arboles_cercanos(geom):
    posibles = list(sindex.intersection(geom.buffer(radio).bounds))
    cercanos = arboles_proj.iloc[posibles]
    return cercanos.distance(geom).lt(radio).sum()

edges_subset = edges.sample(10, random_state=42).copy()  

tqdm.pandas(desc="Calculando sombra")
edges_subset["num_arboles"] = edges_subset.geometry.progress_apply(contar_arboles_cercanos)

# Calcular peso
edges_subset["peso_25_30"] = w_len * edges_subset["length"] - w_sombra * edges_subset["num_arboles"]

print(edges_subset[["length", "num_arboles", "peso_25_30"]])


Calculando sombra: 100%|██████████| 10/10 [00:00<00:00, 385.62it/s]

                                length  num_arboles  peso_25_30
u           v           key                                    
3536480963  10101947238 0    14.885372            0    2.977074
10873732466 10873732476 0    48.955626            0    9.791125
10248794380 10248794360 0    32.265947            1    5.653189
1636824903  10942249303 1    45.011365            0    9.002273
10908411199 10911332037 0    10.547231            4   -1.090554
11013591667 11013591668 0     9.019294            3   -0.596141
112137514   112137583   0    19.150238            6   -0.969952
11177553400 11177553402 0     5.107707            0    1.021541
4102819109  4102819107  0    54.512278            0   10.902456
11536783077 7195974007  0    25.836072           13   -5.232786


## Generación de columnas de peso para walk y cycle

In [ ]:
#Grafo WALK
radio = 15
tramos = [(0, 5), (5, 10), (10, 15), (15, 20), (20, 25), (25, 30), (30, 35), (35, 40)]
pesos = {
    (0, 5): (0.7, 0.3),
    (5, 10): (0.6, 0.4),
    (10, 15): (0.5, 0.5),
    (15, 20): (0.4, 0.6),
    (20, 25): (0.3, 0.7),
    (25, 30): (0.2, 0.8),
    (30, 35): (0.1, 0.9),
    (35, 40): (0.05, 0.95),
}

sindex = arboles.sindex

for u, v, data in tqdm(graph_walk.edges(data=True)):
    geom = data.get("geometry", None)
    if geom is None:
        #construir un line string del nodo u al nodo v
        u_geom=(
            graph_walk.nodes[u]["y"], graph_walk.nodes[u]["x"]
        )
        v_geom=(
            graph_walk.nodes[v]["y"], graph_walk.nodes[v]["x"]
        )
        geom = LineString([u_geom, v_geom])
     
    # Contar árboles cercanos
    posibles = list(sindex.intersection(geom.buffer(radio).bounds))
    cercanos = arboles.iloc[posibles]
    num_arboles = cercanos.distance(geom).lt(radio).sum()

    data["num_arboles"] = num_arboles
    length = data.get("length", 1)

    # Calcular un peso por cada tramo de temperatura
    for tramo in tramos:
        w_len, w_sombra = pesos[tramo]
        t_str = f"peso_{tramo[0]}_{tramo[1]}"
        data[t_str] = w_len * length - w_sombra * num_arboles+1000


  0%|          | 0/126948 [00:00<?, ?it/s]C:\Users\carle\AppData\Local\Temp\ipykernel_17336\2280196184.py:41: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  num_arboles = cercanos.distance(geom).lt(radio).sum()
100%|██████████| 126948/126948 [3:57:29<00:00,  8.91it/s]  


In [ ]:
#guardar el grafo con los pesos
ox.save_graphml(graph_walk, "../data/valencia_walking_sombra.graphml")

In [ ]:
#Grafo CYCLE
radio = 15
tramos = [(0, 5), (5, 10), (10, 15), (15, 20), (20, 25), (25, 30), (30, 35), (35, 40)]
pesos = {
    (0, 5): (0.7, 0.3),
    (5, 10): (0.6, 0.4),
    (10, 15): (0.5, 0.5),
    (15, 20): (0.4, 0.6),
    (20, 25): (0.3, 0.7),
    (25, 30): (0.2, 0.8),
    (30, 35): (0.1, 0.9),
    (35, 40): (0.05, 0.95),
}

sindex = arboles.sindex

for u, v, data in tqdm(graph_cycle.edges(data=True)):
    geom = data.get("geometry", None)
    if geom is None:
        #construir un line string del nodo u al nodo v
        u_geom=(
            graph_cycle.nodes[u]["y"], graph_cycle.nodes[u]["x"]
        )
        v_geom=(
            graph_cycle.nodes[v]["y"], graph_cycle.nodes[v]["x"]
        )
        geom = LineString([u_geom, v_geom])
    

    # Contar árboles cercanos
    posibles = list(sindex.intersection(geom.buffer(radio).bounds))
    cercanos = arboles.iloc[posibles]
    num_arboles = cercanos.distance(geom).lt(radio).sum()

    data["num_arboles"] = num_arboles
    length = data.get("length", 1)

    # Calcular un peso por cada tramo de temperatura
    for tramo in tramos:
        w_len, w_sombra = pesos[tramo]
        t_str = f"peso_{tramo[0]}_{tramo[1]}"
        data[t_str] = w_len * length - w_sombra * num_arboles+1000


In [ ]:
#guardar el grafo con los pesos
ox.save_graphml(graph_cycle, "../data/valencia_cycling_sombra.graphml")